**Libraries Used**

In [1]:
import os
from PIL import Image
import matplotlib.pyplot as plt
import cv2
import numpy as np
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors

**Reading the Images**

In [2]:
def read_images_in_folder(folder_path): #Read images from a folder
    image_list = []
    for filename in os.listdir(folder_path):
        image_path = os.path.join(folder_path, filename)
        img = Image.open(image_path)
        image_list.append(img)

    return image_list  #Returns a list of Images read from the folder 
images = read_images_in_folder("Images")

**Performing K-Means clustering**

Removing of the Clusters for better visualization 

In [ ]:
remove_cluster = []
def remove_cluster_from_image(image, k=2): #Remove the dominant cluster from the given image using KMeans clustering.
    image_array = np.array(image)
    if len(image_array.shape) == 3 and image_array.shape[2] == 3: 
        gray_image = cv2.cvtColor(image_array, cv2.COLOR_RGB2GRAY)
    else:
        gray_image = image_array
    pixels = gray_image.reshape((-1, 1))
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(pixels)
    labels = kmeans.labels_
    cluster_to_remove = np.argmax(np.bincount(labels))# Find the most dominant cluster label
    mask = (labels == cluster_to_remove).reshape(gray_image.shape)
    removed_image = np.copy(image)
    removed_image[mask] = [255, 255, 255] 
    edges_removed = cv2.Canny(removed_image, 30, 100) # Detect edges in the removed cluster image using Canny edge detection
    # Plotting the original image, grayscale image, removed cluster image, and edges on removed cluster image
    plt.figure(figsize=(15, 5))

    plt.subplot(1, 4, 1)
    plt.imshow(image)
    plt.title("Original Image")
    plt.axis("off")

    plt.subplot(1, 4, 2)
    plt.imshow(gray_image, cmap='gray')
    plt.title(f"Grayscale Image")
    plt.axis("off")

    plt.subplot(1, 4, 3)
    plt.imshow(removed_image)
    plt.title(f"Removed Cluster (k={k})")
    plt.axis("off")

    plt.subplot(1, 4, 4)
    plt.imshow(edges_removed, cmap='gray')
    plt.title("Edges on Removed Cluster Image")
    plt.axis("off")
    plt.show()
    return edges_removed
for image in images:
    remove_cluster.append(remove_cluster_from_image(image, k=2))


**DBSCAN**

In [5]:
# Set the epsilon and min_samples parameters for DBSCAN
epsilon = 0.001
min_samples = 20

def dbscan_on_images(images, eps, min_samples): #Apply DBSCAN clustering on a list of images.
    clustered_images = []
    for image in images:
        image_array = np.array(image)
        flattened_image = image_array.reshape(-1, 3)
        dbscan = DBSCAN(eps=eps, min_samples=min_samples)
        dbscan.fit(flattened_image)
        labels = dbscan.labels_.reshape(image_array.shape[:2])
        clustered_images.append(labels)

    return clustered_images # Return the list of clustered images
clustered_images = dbscan_on_images(images, eps=epsilon, min_samples=min_samples)



Removing a range of cluster classes after classifiction

In [ ]:
def remove_clusters(image, clusters_to_remove): #Remove specified clusters from the input image.
    mask = np.isin(image, clusters_to_remove)
    image[mask] = 0 

    return image
modified_image_list = []
for idx, (original_image, clustered_image) in enumerate(zip(images, clustered_images)):
    # Plotting the original image, Clustered Image, removed cluster image
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))

    axes[0].imshow(original_image)
    axes[0].set_title('Original Image')
    axes[0].axis('off')

    axes[1].imshow(clustered_image, cmap='viridis') 
    axes[1].set_title(f'Clustered Image {idx + 1}')
    axes[1].axis('off')

    clusters_to_remove = np.arange(12500, 40000)
    modified_image = remove_clusters(np.copy(clustered_image), clusters_to_remove)
    modified_image_list.append(modified_image)
    axes[2].imshow(modified_image)
    axes[2].set_title(f'Modified Image {idx + 1} after Removing Clusters by Number')
    axes[2].axis('off')

    plt.show()


**Bounding Boxes**

In [7]:
def find_contours(drone_image): #Find contours in a given drone image.
    drone_image = np.array(drone_image)
    gray_image = cv2.cvtColor(drone_image, cv2.COLOR_BGR2GRAY)
    blurred_image = cv2.GaussianBlur(gray_image, (5, 5), 0)
    edges = cv2.Canny(blurred_image, 50, 150)
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    return contours,blurred_image,edges,drone_image # Return detected contours, blurred image, edges, and original drone image

**Find bounding boxes formed after edge detection**

In [ ]:
def process_and_display_images(drone_image):  #Process and display images with detected contours.
    contours,blurred_image,edges,drone_image = find_contours(drone_image)
    detected_trees = []
    o_image = drone_image.copy()
    for contour in contours:
        area = cv2.contourArea(contour)
        if area > 500:
            x, y, w, h = cv2.boundingRect(contour)
            detected_trees.append((x, y, x + w, y + h))
    for tree in detected_trees:  # Draw rectangles around detected trees on the original image
        x1, y1, x2, y2 = tree
        cv2.rectangle(drone_image, (x1, y1), (x2, y2), (0, 255, 0), 2)
    contour_image = np.zeros_like(drone_image)
    cv2.drawContours(contour_image, contours, -1, (0, 255, 0), 2) # Draw contours on the contour image
    #Displays the original image, blurred image, edges, and edges with contours overlaid.
    fig, axs = plt.subplots(1, 4, figsize=(15, 5))

    axs[0].imshow(cv2.cvtColor(o_image, cv2.COLOR_BGR2RGB))
    axs[0].set_title('Original Image')
    axs[0].axis('off')

    axs[1].imshow(blurred_image, cmap='gray')
    axs[1].set_title('Blurred Image')
    axs[1].axis('off')

    axs[2].imshow(cv2.cvtColor(edges, cv2.COLOR_GRAY2RGB))
    axs[2].imshow(contour_image, alpha=0.5)
    axs[2].set_title('Edges')
    axs[2].axis('off')

    axs[3].imshow(drone_image)
    axs[3].set_title('Edges with Contours')
    axs[3].axis('off')

    plt.tight_layout()
    plt.show()
for image in images:
    process_and_display_images(image)


**Find bounding boxes formed after edge detection on images that have been clustered by DBSCAN**

In [ ]:
def process_and_display_images(clustered_images): # Process and display images with detected contours and clustered regions.
    i = -1
    for clustered_image in clustered_images:
        i+=1
        clustered_image_array = np.array(clustered_image)
        normalized_image = cv2.normalize(clustered_image_array, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
        clustered_image_rgb = cv2.cvtColor(normalized_image, cv2.COLOR_GRAY2RGB)
        gray_image = cv2.cvtColor(clustered_image_rgb, cv2.COLOR_RGB2GRAY)
        blurred_image = cv2.GaussianBlur(gray_image, (5, 5), 0)
        edges = cv2.Canny(blurred_image, 50, 150)# Detect edges using Canny edge detection
        contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        detected_trees = []
        o_image = np.array(images[i])

        for contour in contours: # Iterate over contours to detect trees and draw rectangles around them
            area = cv2.contourArea(contour)
            if area > 1000:
                x, y, w, h = cv2.boundingRect(contour)
                detected_trees.append((x, y, x + w, y + h))

        for tree in detected_trees: # Draw rectangles around detected trees on the original image
            x1, y1, x2, y2 = tree
            cv2.rectangle(o_image, (x1, y1), (x2, y2), (0, 255, 0), 2)

        contour_image = np.zeros_like(o_image)
        cv2.drawContours(contour_image, contours, -1, (0, 255, 0), 2)
        #Displays the clustered image, blurred image, edges with contours overlaid, and detected trees.
        fig, axs = plt.subplots(1, 4, figsize=(15, 5))

        axs[0].imshow(clustered_image_rgb)
        axs[0].set_title('Clustered Image')
        axs[0].axis('off')

        axs[1].imshow(blurred_image, cmap='gray')
        axs[1].set_title('Blurred Image')
        axs[1].axis('off')

        axs[2].imshow(edges, cmap='gray')
        axs[2].imshow(contour_image, alpha=0.5)
        axs[2].set_title('Edges with Contours')
        axs[2].axis('off')

        axs[3].imshow(o_image)
        axs[3].set_title('Detected Trees')
        axs[3].axis('off')

        plt.tight_layout()
        plt.show()
process_and_display_images(modified_image_list)


**Find bounding boxes formed after edge detection on images that have been clustered by K-Means**

In [ ]:
def process_and_display_images(clustered_image,i): #Process and display images with detected contours and clustered regions.
    clustered_image_array = np.array(clustered_image)
    normalized_image = cv2.normalize(clustered_image_array, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    clustered_image_rgb = cv2.cvtColor(normalized_image, cv2.COLOR_GRAY2RGB)
    gray_image = cv2.cvtColor(clustered_image_rgb, cv2.COLOR_RGB2GRAY)
    blurred_image = cv2.GaussianBlur(gray_image, (5, 5), 0)
    edges = cv2.Canny(blurred_image, 50, 150)
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    detected_trees = []
    o_image = np.array(images[i])

    for contour in contours:
        area = cv2.contourArea(contour)
        if area > 5000: # Threshold for minimum contour area to consider as a tree  
            x, y, w, h = cv2.boundingRect(contour)
            detected_trees.append((x, y, x + w, y + h))

    for tree in detected_trees: # Draw rectangles around detected trees on the original image
        x1, y1, x2, y2 = tree
        cv2.rectangle(o_image, (x1, y1), (x2, y2), (0, 255, 0), 2)

    contour_image = np.zeros_like(o_image)
    cv2.drawContours(contour_image, contours, -1, (0, 255, 0), 2)
    #Displays the clustered image, blurred image, edges with contours overlaid, and detected trees.
    fig, axs = plt.subplots(1, 4, figsize=(15, 5))

    axs[0].imshow(clustered_image_rgb)
    axs[0].set_title('Clustered Image')
    axs[0].axis('off')

    axs[1].imshow(blurred_image, cmap='gray')
    axs[1].set_title('Blurred Image')
    axs[1].axis('off')

    axs[2].imshow(edges, cmap='gray')
    axs[2].imshow(contour_image, alpha=0.5)
    axs[2].set_title('Edges with Contours')
    axs[2].axis('off')

    axs[3].imshow(o_image)
    axs[3].set_title('Detected Trees')
    axs[3].axis('off')

    plt.tight_layout()
    plt.show()
i = 0
for image in remove_cluster:
    
    process_and_display_images(image,i)
    i+=1
    
